In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import numpy as np
import matplotlib.pyplot as plt
from torchvision.models import resnet18
import json

import os
os.chdir('/home/raid/Desktop/Shubh/DLProject/experiments')

from helpers.trainer import Trainer
from helpers.train_all import TrainerAll

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
def whitening_transform(data):
    # Center the data (subtract mean)
    mean = torch.mean(data, dim=0, keepdim=True)
    centered_data = data - mean

    # Compute covariance matrix
    covariance_matrix = torch.matmul(centered_data.T, centered_data) / (centered_data.size(0) - 1)

    # Compute eigenvalues and eigenvectors of the covariance matrix
    eigenvalues, eigenvectors = torch.eig(covariance_matrix, eigenvectors=True)

    # Whitening transformation
    whitened_data = torch.matmul(torch.matmul(eigenvectors, torch.diag(1 / torch.sqrt(eigenvalues[:, 0]))), eigenvectors.T) @ centered_data.T
    
    return whitened_data.T

In [4]:
# Defining batch size as given on page 7 on Adam Research Paper
batch_size = 128    

# Calculate the number of workers to use for DataLoader
num_workers = 8

# transformations
transform = transforms.Compose([transforms.ToTensor()])

In [5]:
os.chdir('/home/raid/Desktop/Shubh/DLProject/experiments/models')

# Load CIFAR-10 dataset
trainset = datasets.CIFAR10(root='../../datasets/', train=True, download=True, transform=transform)
testset = datasets.CIFAR10(root='../../datasets/', train=False, download=True, transform=transform)

# trainset = whitening_transform(trainset)
# testset = whitening_transform(testset)

# Create DataLoader
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

Files already downloaded and verified
Files already downloaded and verified


In [6]:
class ResNet(nn.Module):
    
    def __init__(self):
        super(ResNet, self).__init__()
        self.resnet = resnet18(pretrained=False)
        self.resnet.fc = nn.Linear(512, 10)  # Change output size to match CIFAR-10 classes

    def forward(self, x):
        x = self.resnet(x)
        return x

In [7]:
criterion = nn.CrossEntropyLoss()
epochs = 200 

optimizer = optim.Adam

In [9]:
model = ResNet().to(device)
trainer = TrainerAll(train_loader, test_loader, criterion, epochs=10)

head1, head2, head3 = 'EPOCHS', 'TRAIN LOSS', 'VAL LOSS'
print(f'{head1 :>6} {head2 :>25} {head3 :>25}')

log = trainer.train_all_optim(model)

EPOCHS                TRAIN LOSS                  VAL LOSS
Optimizer : Adam 4
ot:  tensor([[-2.3238, -1.3336, -2.5218,  ...,  2.0036, -0.6383,  3.1698],
        [-0.2533, -1.2251, -2.5264,  ..., -0.1630,  0.5435,  2.9064],
        [-0.8074, -3.9373, -2.1068,  ..., -0.8173,  0.2497,  0.5880],
        ...,
        [ 1.4371, -1.6890, -0.7536,  ...,  1.5196,  0.5616,  0.8728],
        [-2.4982, -1.6914,  0.9545,  ...,  0.9302, -0.3962, -0.2494],
        [ 0.1659, -1.1065, -0.2832,  ..., -1.3920,  0.5868,  0.7826]],
       device='cuda:0', grad_fn=<AddmmBackward0>)
it:  tensor([[[[0.7255, 0.6980, 0.7059,  ..., 0.7373, 0.7608, 0.7725],
          [0.7176, 0.7059, 0.7137,  ..., 0.7333, 0.7098, 0.7255],
          [0.7451, 0.7451, 0.7608,  ..., 0.7451, 0.7020, 0.7216],
          ...,
          [0.6863, 0.6510, 0.6745,  ..., 0.7725, 0.6549, 0.6941],
          [0.6510, 0.6275, 0.6706,  ..., 0.6627, 0.5725, 0.6353],
          [0.6471, 0.6471, 0.6824,  ..., 0.5922, 0.6000, 0.6235]],

         [[0.66

KeyboardInterrupt: 